In [ ]:
import shutil
import subprocess
from dotenv import load_dotenv
from pydantic_ai import Agent
import yaml
from workout_builder.py.workout_definition import WorkoutDefinition
from pathlib import Path
import logging
import re
load_dotenv(override=True)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


async def generate_workout(workout_description: str, model: str, use_structured_output: bool) -> WorkoutDefinition:
    schema = WorkoutDefinition.model_json_schema()
    
    system_prompt = """You are a helpful assistant that helps translate user requests into structured workout definitions.
    Also give the workout a creative name and description."""
    
    if not use_structured_output:
        system_prompt = f"""
        {system_prompt}.
        Use the json schema for the output:
        ```
        {schema}
        ```
        
        """
        agent = Agent(model=model)
        prompt = f"""Help me generate a json file for the workout: {workout_description}"""
        result = await agent.run(system_prompt + prompt)

        # extract text between markdown code blocks
        match = re.search(r"```(json)?\n(.*?)\n```", result.output, re.DOTALL)
        if match:
            json_text = match.group(2)
            result.output = json_text
        else:
            print("No code block found")

        workout: WorkoutDefinition = WorkoutDefinition.model_validate_json(json_text)
    else:
        agent = Agent(model=model, system_prompt=system_prompt)
        prompt = f"""Help me generate a workout for: {workout_description}"""
        result = agent.run_sync(prompt,output_type=WorkoutDefinition)

        workout = result.output

    print(f"💡 Created workout: {workout.metadata.name}")
    return workout


def _sanitize_name(n: str) -> str:
    # Mirror Java sanitizeName: keep A-Za-z0-9 _- then trim length 30 then replace spaces with '_' for filename
    cleaned = re.sub(r"[^A-Za-z0-9 _-]", "", n)[:30]
    return cleaned.replace(" ", "_")

def encode_to_fit(yaml_file: str, fit_file: str):
    """Encode a YAML workout to FIT using Java encoder without passing an explicit name.
    The Java program will read metadata.name. We post-compute expected filename and move it to fit_file."""
    JAVA_DIR = Path(
        "/Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/java"
    )
    JAVA_BUILD = JAVA_DIR / "build"
    JAVA_LIB_DIR = JAVA_DIR / "lib"
    FIT_JAR = JAVA_LIB_DIR / "fit.jar"
    SNAKEYAML_JAR = JAVA_LIB_DIR / "snakeyaml-2.2.jar"
    CLASSPATH = ":".join([str(JAVA_BUILD), str(FIT_JAR), str(SNAKEYAML_JAR)])
    ENCODER_CLASS = "com.neuraltag.workout.EncodeYamlWorkout"

    # Parse YAML to discover metadata.name for expected filename
    with open(yaml_file, "r") as yf:
        data = yaml.safe_load(yf)
    meta_name = data.get("metadata", {}).get("name", "Workout")
    produced_filename = f"{_sanitize_name(meta_name)}.fit"

    command = f"java -cp {CLASSPATH} {ENCODER_CLASS} {yaml_file}"
    subprocess.run(command, shell=True, check=True)

    # Move generated FIT to desired location
    shutil.move(produced_filename, fit_file)
    logger.info(f"Encoded workout using metadata name '{meta_name}' -> {fit_file}")



In [ ]:

logger.info("Starting workout generation")
input = "5 min warmup + 5 x (30s@4:20 + 30s rest) + 6x[1km@(3:45-3:50) + 1 min rest) ] + 15 min cool down @ 6:00"

logger.info(f"Converting `{input}` to yaml")
workout = await generate_workout(
    workout_description=input,
    # model="google-gla:gemma-3-27b-it",
    model = "google-gla:gemini-2.5-flash-lite",
    use_structured_output=False,
)
print(yaml.dump(workout.model_dump(mode="json", exclude_none=True),sort_keys=False))


In [ ]:
name = workout.metadata.name.replace(" ", "_").lower()
yaml_file = f"/Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/{name}.yaml"
fit_file = f"/Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/{name}.fit"
with open(yaml_file, "w") as f:
    yaml.dump(workout.model_dump(mode="json", exclude_none=True), f,sort_keys=False,indent=4)

# Encode without passing name (Java reads metadata.name)
encode_to_fit(yaml_file, fit_file)
logger.info(f"💾 Wrote workout to {yaml_file}\nand {fit_file}")


INFO:__main__:💾 Wrote workout to /Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/the_endurance_builder.yaml
and /Users/niklasvonmaltzahn/Documents/personal/neuraltag/workout_builder/examples/the_endurance_builder.fit


Wrote the_endurance_builder.fit
Created FIT workout with 8 steps (including repeat controllers).
